<a href="https://colab.research.google.com/github/tpmarsha/ML2AmazonKaggle/blob/master/Category_Prediction_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip uninstall tensorflow==2.2.0
!pip install tensorflow==1.15.0
!pip install bert-tensorflow
import pandas as pd
import os
import time
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow.compat.v1 as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, Bidirectional, LSTM
import tensorflow_hub as hub
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling


In [0]:
def read_json(f):
  for l in open(f,"r"):
    yield eval(l)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
root_path = '/content/drive/My Drive/Team 3 Kaggle Competition ML2/'
df= pd.read_csv(root_path + 'train_CategoryPrediction.csv',
               index_col=0)
df.head(5)


,ReviewHash,ReviewText,CategoryLabel
0,R798569390,The model in this picture has them rolled up a...,Women
1,R436443063,"I love the look of this bra, it is what I want...",Women
2,R103439446,I am not comfortable with wearing my wedding b...,Women
3,R486351639,Like the look of this top and really looks cut...,Women
4,R508664275,I'm quite small and the XS fits me like a regu...,Women


In [0]:
display(df['CategoryLabel'].value_counts())

Women    141303
Men       51530
Baby       3324
Girls      2191
Boys       1652
Name: CategoryLabel, dtype: int64

In [0]:
Replace_by_space_re = re.compile('[/(){}\[\]\|@,;]')
Bad_symbols_re = re.compile('[^0-9a-z #+_]')
# Stopwords = set(stopwords.words('english'))

def clean_text(text):
    text=str(text)
    text=text.lower()
    text=Replace_by_space_re.sub(' ', text)
    text=Bad_symbols_re.sub('', text)
    # text= ' '.join(word for word in text.split() if word not in Stopwords)
    return text

df['ReviewText'] = df['ReviewText'].apply(clean_text)
print(df.head(5))
# ### Word Size
# print(df['ReviewText'].apply(lambda x: len(x.split(' '))).sum())
# ### Max Review Length
# print(df['ReviewText'].apply(lambda x: len(x.split(' '))).max())

# df['tokens_length'] = df['ReviewText'].apply(lambda x: len(x.split(' ')))
# df[df.tokens_length==2046].ReviewText


   ReviewHash                                         ReviewText CategoryLabel
0  R798569390  the model in this picture has them rolled up a...         Women
1  R436443063  i love the look of this bra  it is what i want...         Women
2  R103439446  i am not comfortable with wearing my wedding b...         Women
3  R486351639  like the look of this top and really looks cut...         Women
4  R508664275  im quite small and the xs fits me like a regul...         Women


In [0]:
X =df.ReviewText
print(X.head(5))
y= df.CategoryLabel
print(y.unique)
train, test, y_train, y_test = train_test_split(df, y, test_size=0.1, random_state = 42, stratify=y)

0    the model in this picture has them rolled up a...
1    i love the look of this bra  it is what i want...
2    i am not comfortable with wearing my wedding b...
3    like the look of this top and really looks cut...
4    im quite small and the xs fits me like a regul...
Name: ReviewText, dtype: object
<bound method Series.unique of 0         Women
1         Women
2         Women
3         Women
4         Women
          ...  
199995    Women
199996    Women
199997    Women
199998    Women
199999     Baby
Name: CategoryLabel, Length: 200000, dtype: object>


In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()
tokenizer.tokenize("This here's an example of using the BERT tokenizer")



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [0]:
GUID = 'ReviewHash'
DATA_COLUMN = 'ReviewText'
LABEL_COLUMN = 'CategoryLabel'

# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [0]:
label_list = ['Men', 'Women',"Boys",'Girls','Baby']
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 180000


INFO:tensorflow:Writing example 0 of 180000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: Men (id = 0)


INFO:tensorflow:label: Men (id = 0)


INFO:tensorflow:Writing example 10000 of 20000


INFO:tensorflow:Writing example 10000 of 20000


In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        # auc = tf.metrics.auc(
        #     label_ids,
        #     predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            # "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 2000
SAVE_SUMMARY_STEPS = 500

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
print(num_train_steps)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)


16875


In [0]:
# Specify outpit directory and number of checkpoint steps to save
OUTPUT_DIR = root_path
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/Team 3 Kaggle Competition ML2/', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 2000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f54c8569a58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/Team 3 Kaggle Competition ML2/', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 2000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f54c8569a58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [0]:
from datetime import datetime
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 16875 into /content/drive/My Drive/Team 3 Kaggle Competition ML2/model.ckpt.


INFO:tensorflow:Saving checkpoints for 16875 into /content/drive/My Drive/Team 3 Kaggle Competition ML2/model.ckpt.


INFO:tensorflow:Loss for final step: 0.16582648.


INFO:tensorflow:Loss for final step: 0.16582648.


Training took time  2:14:09.210256


In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 16875: eval_accuracy = 0.86435, false_negatives = 1041.0, false_positives = 1346.0, global_step = 16875, loss = 0.5003941, precision = 0.91116685, recall = 0.92988485, true_negatives = 3807.0, true_positives = 13806.0


INFO:tensorflow:Saving dict for global step 16875: eval_accuracy = 0.86435, false_negatives = 1041.0, false_positives = 1346.0, global_step = 16875, loss = 0.5003941, precision = 0.91116685, recall = 0.92988485, true_negatives = 3807.0, true_positives = 13806.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 16875: /content/drive/My Drive/Team 3 Kaggle Competition ML2/model.ckpt-16875


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 16875: /content/drive/My Drive/Team 3 Kaggle Competition ML2/model.ckpt-16875


{'eval_accuracy': 0.86435,
 'false_negatives': 1041.0,
 'false_positives': 1346.0,
 'global_step': 16875,
 'loss': 0.5003941,
 'precision': 0.91116685,
 'recall': 0.92988485,
 'true_negatives': 3807.0,
 'true_positives': 13806.0}

In [0]:
def getPrediction(in_sentences):
  labels = ['Men','Women','Girls','Boys', 'Baby']
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 'Men') for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [labels[prediction['labels']] for prediction in list(predictions)]
  

In [0]:
pred_sentences = [
  'I love this blouse, in fact I have it on right now....all of my friends like it and want one.',
  "Cute product. Loved the fit. Fast shipping! I would recommended breaking it in before wearing. Hurts a bit on your first few wears. I also recommend wearing sandal socks cause your toes hurt a bit in these shoes.",
  "I wanted a formal watch that had a big face and this definitely was the coolest watch out there with a great price! I have gotten many compliments on this watch. You will have to go to a jeweler or watch store to get it sized once you get it. My only complaint is that the turn dial on the face makes squeaky noises when you move around the watch.'",
  "These gladiator sandals fit true to size and arrived right on time. They're also true to color and came exactly as pictured here. However, the calf area was too narrow for me, and would better suit someone with slimmer calves. I had to squeeze my calves, (which really aren't big at all, just not small enough to comfortably fit) into the shoe and zip it in. It's a bit of a tight fit but not unbearable or irritating. And that's the only reason why I'm giving four stars. Otherwise, it's comfy in the foot area."
]

In [0]:
getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


['Women', 'Women', 'Men', 'Women']

In [0]:
### Printing Prediction for Submission on Kaggle
def get_test_text():
  input_text = []
  for l in read_json(root + "test_Category.json"):
    reviewerID,reviewerHash,reviewText = l['reviewerID'],l['reviewHash'],l['reviewText'].lower()
    input_text.append([reviewerID,reviewerHash,reviewText])
  df = pd.DataFrame(input_text)
  df.columns = ['reviewerID','reviewHash','reviewText']
  return df
  
df = get_test_text()
df.head(5)
df['reviewText'] = df['reviewText'].apply(clean_text)

In [0]:
pred_sentences = df['reviewText']. values. tolist()
pred_labels = getPrediction(pred_sentences)
print(len(pred_labels))


In [0]:
catDict = {
  "Women": 0,
  "Men": 1,
  "Girls": 2,
  "Boys": 3,
  "Baby": 4
}
pred_labels = [catDict[i] for i in pred_labels] #Convert to submission mapping
# pred_labels[:5]


In [0]:
pred_df = pd.DataFrame(pred_labels)
pred_df.columns = ['predictionLabel']
submission = pd.concat([df,pred_df],axis=1)
submission.head(5)

,reviewerID,reviewHash,reviewText,predictionLabel
0,U281659737,R934811302,i love this blouse in fact i have it on right...,0
1,U670561057,R657711680,cute product loved the fit fast shipping i wou...,0
2,U433746872,R750304163,i wanted a formal watch that had a big face an...,1
3,U327816997,R865011815,my daughter used this dress for her first comm...,0
4,U323131234,R222729968,nice shirt good quality with pockets kind of...,1


In [0]:
def print_predictions(representation,classifier,data):
  root = '/content/drive/My Drive/Team 3 Kaggle Competition ML2/'
  input_representation = representation
  predictions = open(root + "prediction_Category_" + input_representation + '_'+ classifier + ".txt", 'w')
  predictions.write("reviewerID-reviewHash,category\n")
  for index, row in data.iterrows():
    # print(row['reviewerID'])
    # break
    predictions.write(str(row['reviewerID']) + '-' + str(row['reviewHash']) + ',' + str(row['predictionLabel']) + "\n")
  predictions.close()

print_predictions('BERT', 'classifier', submission)

# New Section